# Disk Artifacts in memory
Even though memory has several artifacts not found on disk it contains several traditionally disk based forensics artifacts that can be parsed and analyzed without taking a full disk image. The artifacts covered in this lesson are the following:

* Event logs
* $MFT
* Prefetch

## Event Logs

Event logs house numerous artifact for incident response. The three main Windows event logs are the following:

* Security
* System
* Application

There are many more event logs then these and any applicaiton can create it's own event log. More recent relavent event logs are Powershell and Sysmon specific event logs which keep track of powershell command exection and Sysmon is an event log which can be customized to gather network and process exectuion similar to using the `sysmon` tool in malware analysis. Windows logs are stored in these directories on disk:

* Application - %SYSTEMROOT%/System32\winevt\Logs\application.evtx
* System - %SYSTEMROOT%/System32\winevt\Logs\system.evtx
* Security - %SYSTEMROOT%/System32\winevt\Logs\security.evtx

Security event logs are where user logon and logoff activity are stored as well as if configured correctly process execution. Security event logs events are numerous but the main events to monitor for related to logon events are the following:

|Event ID| Description|
|--------|------------|
| 4624   |An account was successfully logged on|
|4625    | An account failed to log on|
|4778    | A session was reconnected to a Window Station|
|4779    | A session was disconnected from a Window Station|

There are logon types assocaited with these events which will tell how the logon happed weather it be through an interactive session or network. A list of common logon types are listed below:

|Event ID| Description|
|--------|------------|
|2| Interactive|
|3|Network|
|4|Batch|
|5|Service|
|6|Proxy|
|7|Unlock|
|8|Network Cleartext|
|9|New Credentials|
|10|Remote Interactive|
|11|Cache Interactive|

All of these types have specific meaning and are described below:

* Interactive – logged on from console, keyboard, runas or KVM
* Network – net use or IIS integrated auth
* Batch – scheduled task
* Service – service logged on with configured credintials
* Proxy – not sure
* Unlock – system unlocked from screensaver
* NetworkClearText – logons from web servers
* NewCredintials – Assumed through runas
* RemoteInteractive – logged on through terminal services, remote assistance or RDP
* CacheInteractive – cached 

There are other logon events related to Kerbos that are also relavent that have different Event ID's. These specific events are summarized below:

|Event ID|Description|
|--------|-----------|
|4772 | A Kerberos authentication ticket request failed|
|4768 | A Kerberos authentication ticket (TGT) was requested|
|4769| A Kerberos service ticket was requested|
|4770|A Kerberos service ticket was renewed|
|4773|A Kerberos service ticket request failed|
|4771|Kerberos pre-authentication failed|
|4768|A Kerberos authentication ticket (TGT) was requested|
|4774|An account was mapped for logon|
|4776|The domain controller attempted to validate the credentials for an account|
|4778|A session was reconnected to a Window Station|
|4779 |A session was disconnected from a Window Station|

Several other event ID's are not related to account logon or logoff but related to account managment, these event ID's are summarized below:

|Event ID|Description|
|--------|-----------|
|4720| A user account was created|
|4722| A user account was enabled|
|4723| An attempt was made to change an account's password|
|4724| An attempt was made to reset an accounts password|
|4725| A user account was disabled|
|4726| A user account was deleted|
|4727| A security-enabled global group was created|
|4728| A member was added to a security-enabled global group|
|4729| A member was removed from a security-enabled global group|
|4730| A security-enabled global group was deleted|
|4731|A security-enabled local group was created|
|4732|A member was added to a security-enabled local group|
|4733| A member was removed from a security-enabled local group|
|4734| A security-enabled local group was deleted|
|4735| A security-enabled local group was changed|
|4737| A security-enabled global group was changed|
|4738| A user account was changed|
|4739| Domain Policy was changed|
|4740|A user account was locked out|

System and Application event logs do not have the same consistency of event ID's and descriptions to parse but can contain relavent information for analysis. The Application event log can contain any applications events with custom ID's and descriptions. With the move to Windows 7 many applications started creating their own Windows Event Logs but many still write to the Windows Application event log. The system event log does have some consitent events which are realted to services starting. Event ID's and descritions relatedt o services are located below:

|Event ID|Description|
|--------|-----------|
|7035|Service sent a start contol|
|7036|Service entered the running state|
|7035|Service successfully sent a stop control|
|7036|Service entered the stop state|

Services with these ID's could be malicous backdoors starting as a service or attacker utilities such as `PSEXEC` or Windows Credintial Editor.

Event logs in memory can be parsed in only older memory images. To parse the event log execute the folowing commands and volatility plugin:


In [8]:
mkdir output



    Directory: C:\Users\jsmith\Documents\GitHub\IncidentResponse\diskforensicsinmemoryanalysis


Mode                LastWriteTime         Length Name                                                                  
----                -------------         ------ ----                                                                  
d-----        4/23/2018  10:14 PM                output                                                                




In [9]:
python ../volatility/vol.py -f ../APT.img --profile=WinXPSP2x86 evtlogs -D output

Volatility Foundation Volatility Framework 2.6
Parsed data sent to internet.txt
Parsed data sent to appevent.txt
Parsed data sent to sysevent.txt
Parsed data sent to secevent.txt


In [10]:
cat output/sysevent.txt

2008-08-11 16:06:48 UTC+0000|sysevent.evt|MACHINENAME|N/A|EventLog|6009|Info|5.01.;2600;Service Pack 3;Uniprocessor Free
2008-08-11 16:06:48 UTC+0000|sysevent.evt|MACHINENAME|N/A|EventLog|6005|Info|N/A
2008-08-11 16:07:05 UTC+0000|sysevent.evt|MACHINENAME|N/A|Serial|2|Info|\Device\Serial0;\Device\Serial0
2008-08-11 16:07:05 UTC+0000|sysevent.evt|MACHINENAME|N/A|Serial|2|Info|\Device\Serial1;\Device\Serial1
2008-08-11 20:21:13 UTC+0000|sysevent.evt|DEMO-C442CF8FA3|N/A|EventLog|6011|Info|MACHINENAME;DEMO-C442CF8FA3
2008-08-11 20:24:12 UTC+0000|sysevent.evt|DEMO-C442CF8FA3|N/A|Workstation|3260|Info|workgroup;WORKGROUP
2008-08-11 20:25:47 UTC+0000|sysevent.evt|DEMO-C442CF8FA3|N/A|HTTP|15007|Info|;http://*:2869/
2008-08-11 20:31:14 UTC+0000|sysevent.evt|DEMO-C442CF8FA3|N/A|Setup|60054|Info|2600
2008-08-11 21:02:27 UTC+0000|sysevent.evt|DEMO-C442CF8FA3|N/A|EventLog|6009|Info|5.01.;2600;Service Pack 3;Uniprocessor Free
2008-08-11 21:02:27 UTC+0000|sysevent.evt|DEMO-C442CF8FA3|N/A|EventLog|600

In [12]:
cat output/appevent.txt

2008-08-11 20:21:15 UTC+0000|appevent.evt|DEMO-C442CF8FA3|N/A|LoadPerf|1000|Info|RSVP;QoS RSVP
2008-08-11 20:21:16 UTC+0000|appevent.evt|DEMO-C442CF8FA3|N/A|LoadPerf|1000|Info|PSched;PSched
2008-08-11 20:21:20 UTC+0000|appevent.evt|DEMO-C442CF8FA3|N/A|LoadPerf|1000|Info|RemoteAccess;Routing and Remote Access
2008-08-11 20:24:41 UTC+0000|appevent.evt|DEMO-C442CF8FA3|N/A|LoadPerf|1000|Info|TermService;Terminal Services
2008-08-11 20:24:43 UTC+0000|appevent.evt|DEMO-C442CF8FA3|N/A|LoadPerf|1000|Info|MSDTC;MSDTC
2008-08-11 20:24:44 UTC+0000|appevent.evt|DEMO-C442CF8FA3|N/A|MSDTC|4104|Info|N/A
2008-08-11 20:24:44 UTC+0000|appevent.evt|DEMO-C442CF8FA3|N/A|MSDTC|2444|Info|0;0;0;0;0;0
2008-08-11 20:24:59 UTC+0000|appevent.evt|DEMO-C442CF8FA3|N/A|LoadPerf|1000|Info|WmiApRpl;WmiApRpl
2008-08-11 20:25:00 UTC+0000|appevent.evt|DEMO-C442CF8FA3|N/A|LoadPerf|1001|Info|WmiApRpl;WmiApRpl
2008-08-11 20:25:00 UTC+0000|appevent.evt|DEMO-C442CF8FA3|N/A|LoadPerf|1000|Info|WmiApRpl;WmiApRpl
2008-08-11 20:25:

In [15]:
cat output/secevent.txt

In [16]:
cat output/internet.txt

You can also use this command to dump out the native `EVT` files with the option `--save-evt`. On newer operating systems you can dump the `EVTX` files using the dumpfiles command using the regex option `dumpfiles -D output/ -r evtx$ -i -S summary.txt`. These files can then be opened with other event log parsing tools such as Event Log Explorer or python-evtx. 